In [4]:
import pandas as pd
import numpy as np
from glmnet import LogitNet
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.datasets import load_svmlight_files
from sklearn.metrics import accuracy_score
import xgboost as xgb
import csv
from xgboost import XGBRegressor
from sklearn import preprocessing
import seaborn as sns
import math
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import Imputer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler

### Computer system spec
MacBook Air (13-inch, Early 2014)

Processor 1.4 GHz Intel Core i5

Memory 4 GB 1600 MHz DDR3

### Load the dataset Ames_data.csv

In [5]:
# read csv file
df = pd.read_csv('/Users/glen/Desktop/Statistical learning/Project/Ames_data.csv')
df.head(5)

,PID,MS_SubClass,MS_Zoning,Lot_Frontage,Lot_Area,Street,Alley,Lot_Shape,Land_Contour,Utilities,...,Fence,Misc_Feature,Misc_Val,Mo_Sold,Year_Sold,Sale_Type,Sale_Condition,Longitude,Latitude,Sale_Price
0,526301100,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,141,31770,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,...,No_Fence,None,0,5,2010,WD,Normal,-93.619754,42.054035,215000
1,526350040,One_Story_1946_and_Newer_All_Styles,Residential_High_Density,80,11622,Pave,No_Alley_Access,Regular,Lvl,AllPub,...,Minimum_Privacy,None,0,6,2010,WD,Normal,-93.619756,42.053014,105000
2,526351010,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,81,14267,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,...,No_Fence,Gar2,12500,6,2010,WD,Normal,-93.619387,42.052659,172000
3,526353030,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,93,11160,Pave,No_Alley_Access,Regular,Lvl,AllPub,...,No_Fence,None,0,4,2010,WD,Normal,-93.617320,42.051245,244000
4,527105010,Two_Story_1946_and_Newer,Residential_Low_Density,74,13830,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,...,Minimum_Privacy,None,0,3,2010,WD,Normal,-93.638933,42.060899,189900


In [6]:
# Descriptive statistics for each column
df.describe()

,PID,Lot_Frontage,Lot_Area,Year_Built,Year_Remod_Add,Mas_Vnr_Area,BsmtFin_SF_1,BsmtFin_SF_2,Bsmt_Unf_SF,Total_Bsmt_SF,...,Enclosed_Porch,Three_season_porch,Screen_Porch,Pool_Area,Misc_Val,Mo_Sold,Year_Sold,Longitude,Latitude,Sale_Price
count,2.930000e+03,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,...,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000
mean,7.144645e+08,57.647782,10147.921843,1971.356314,1984.266553,101.096928,4.177474,49.705461,559.071672,1051.255631,...,23.011604,2.592491,16.002048,2.243345,50.635154,6.216041,2007.790444,-93.642897,42.034482,180796.060068
std,1.887308e+08,33.499441,7880.017759,30.245361,20.860286,178.634545,2.233372,169.142089,439.540571,440.968018,...,64.139059,25.141331,56.087370,35.597181,566.344288,2.714492,1.316613,0.025700,0.018410,79886.692357
min,5.263011e+08,0.000000,1300.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,-93.693153,41.986498,12789.000000
25%,5.284770e+08,43.000000,7440.250000,1954.000000,1965.000000,0.000000,3.000000,0.000000,219.000000,793.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000,-93.660217,42.022088,129500.000000
50%,5.354536e+08,63.000000,9436.500000,1973.000000,1993.000000,0.000000,3.000000,0.000000,465.500000,990.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,-93.641806,42.034662,160000.000000
75%,9.071811e+08,78.000000,11555.250000,2001.000000,2004.000000,162.750000,7.000000,0.000000,801.750000,1301.500000,...,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,-93.622113,42.049853,213500.000000
max,1.007100e+09,313.000000,215245.000000,2010.000000,2010.000000,1600.000000,7.000000,1526.000000,2336.000000,6110.000000,...,1012.000000,508.000000,576.000000,800.000000,17000.000000,12.000000,2010.000000,-93.577427,42.063388,755000.000000


In [7]:
# df shape
testnum = int(round(df.shape[0]*0.3 , 0))
print(df.shape)
print("Test number:", testnum)

#make sure PID is unique
print("unique PID number",pd.unique(df['PID']).size)

(2930, 83)
Test number: 879
unique PID number 2930


### Split the data into 70% training and 30% testing dataset

In [8]:
# Split data in input and output
original_y = df['Sale_Price']
original_X = df.drop(columns = ['Sale_Price'])

#Split data in train and test set
X_train, X_test, y_train, y_test = train_test_split(original_X, original_y, test_size=0.3,random_state=12)

print("Train dataset contains {0} rows and {1} columns".format(X_train.shape[0], X_train.shape[1]))
print("Test dataset contains {0} rows and {1} columns".format(X_test.shape[0], X_test.shape[1]))

#Save the train.csv and test.csv file

X_train.to_csv('train.csv', index = False)
X_test.join(y_test).to_csv('test.csv', index = False)

# pd.merge(X_test,y_test,left_on='index',right_on='index')


Train dataset contains 2051 rows and 82 columns
Test dataset contains 879 rows and 82 columns


### Data preprocessing
Because I try to use one-hot encoding later. In order to decrease the number of categories. First I delete the column if its categories exceed 15. Training dataset and test dataset would get different number of columns while doing one-hot encoding, so I need to align the X_train and X_test together to get the same number of column. Then I use imputer to replace the Nan value by the mean value. 

In [9]:
#data preprocessing

#delete column 'Street', 'Longtitude', 'Latitude'
X_train = X_train.drop(columns = ['Street','Longitude','Latitude','Garage_Yr_Blt'])
X_test = X_test.drop(columns = ['Street','Longitude','Latitude','Garage_Yr_Blt'])

#Choose column
choose_column = [col for col in X_train.columns if (X_train[col].nunique() < 15 and X_train[col].dtype == "object") or X_train[col].dtype in ['int64','float64']]

X_train = X_train[choose_column]
X_test = X_test[choose_column]

print("Train dataset contains {0} rows and {1} columns".format(X_train.shape[0], X_train.shape[1]))
print("Test dataset contains {0} rows and {1} columns".format(X_test.shape[0], X_test.shape[1]))

Train dataset contains 2051 rows and 74 columns
Test dataset contains 879 rows and 74 columns


In [10]:
#implement one hot encoding to get the result
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)
print("Train dataset after one hot contains {0} rows and {1} columns".format(X_train.shape[0], X_train.shape[1]))
print("Test dataset after one hot contains {0} rows and {1} columns".format(X_test.shape[0], X_test.shape[1]))

Train dataset after one hot contains 2051 rows and 264 columns
Test dataset after one hot contains 879 rows and 256 columns


In [11]:
# align X_train and X_test so column number of X_train and X_test could be the same

X_train, X_test = X_train.align(X_test, join = 'left', axis=1)
print("Train dataset contains {0} rows and {1} columns".format(X_train.shape[0], X_train.shape[1]))
print("Test dataset contains {0} rows and {1} columns".format(X_test.shape[0], X_test.shape[1]))

Train dataset contains 2051 rows and 264 columns
Test dataset contains 879 rows and 264 columns


### Label Encoder 
I've tried to use label encoder first, but it turns out a bad predicted result compared to one hot encoding.

In [ ]:
###### Label Encoder 

# def label_encoder(dataframe ,col_name):
#     #Use label encoder 
#     le = preprocessing.LabelEncoder()
#     le.fit(dataframe[col_name])

#     #create a new dataframe d then replace the original value
#     test = le.transform(dataframe[col_name]) 
#     d = {'col1': list(test)}
#     df = pd.DataFrame(data=d, dtype=np.int8)
    
#     #Replace original value by label_encoded value
#     dataframe[col_name]  = df.values
#     return None

# # X_train label_encoder transform
# for col in X_train.columns:
#     if X_train[col].dtype == "object":
#         label_encoder(X_train,col)

# # X_test label_encoder transform
# for col in X_test.columns:
#     if X_test[col].dtype == "object":
#         label_encoder(X_test,col)

# print("Train dataset contains {0} rows and {1} columns".format(X_train.shape[0], X_train.shape[1]))
# print("Test dataset contains {0} rows and {1} columns".format(X_test.shape[0], X_test.shape[1]))

In [12]:
#Use Imputer to deal with missing value or nan
#train_X and test_X are numpy array
my_imputer = Imputer(missing_values='NaN', strategy='mean')
train_X = my_imputer.fit_transform(X_train)
test_X = my_imputer.transform(X_test)
print("Train dataset contains {0} rows and {1} columns".format(train_X.shape[0], train_X.shape[1]))
print("Test dataset contains {0} rows and {1} columns".format(test_X.shape[0], test_X.shape[1]))

Train dataset contains 2051 rows and 264 columns
Test dataset contains 879 rows and 264 columns


### Save X_train and y_train in csv files. Then I use glmnet in R to predict the best lambda.  

In [13]:
# save X_train & X_test as csv file which are used to compute best lambda
# best lambda = 763.7335
X_train.fillna(0)
X_train.to_csv('X_train.csv', index = False)
y_train.to_csv('y_train.csv', index = False)

### XGboost
XGBoost is an optimized distributed gradient boosting system designed to be highly efficient, flexible and portable. It implements machine learning algorithms under the Gradient Boosting framework.

In [14]:
#XGboost
XGB = XGBRegressor()
XGB.fit(train_X, y_train, verbose=False)
pre_test_y = XGB.predict(test_X)
ans = round(math.sqrt(np.mean(np.square(np.log(pre_test_y) - np.log(np.array(y_test))))), 4)
print("Root-Mean-Squared-Error (RMSE) for XGBoost model is {0}".format(ans))

Root-Mean-Squared-Error (RMSE) for XGBoost model is 0.1288


In [22]:
# export the result as mysubmission1.txt
PID = np.array(X_test['PID'].values).reshape(len(X_test),1)
Sale_Price = pre_test_y.reshape(len(pre_test_y),1)
ans = np.concatenate((PID,Sale_Price),axis=1)

# save the data with mixted datatype, %d saves PID as integer and %10.5f can save the sale_price as double 
np.savetxt("mysubmission1", ans, delimiter=',', header="PID,Sale_Price", fmt='%d %10.5f', comments='')


### Gradient Boosted Regression
I've tried random forest algorithm in the beginning, but the RMSE was still high. So I use GBR which is similiar to XGboost. 

In [ ]:
# Use Gradient Boosted Regression Trees
gbrt=GradientBoostingRegressor(n_estimators=100) 
gbrt.fit(train_X, y_train) 
pre_test_y=gbrt.predict(test_X)
ans = round(math.sqrt(np.mean(np.square(np.log(pre_test_y) - np.log(np.array(y_test))))), 4)
print("Root-Mean-Squared-Error (RMSE) for GBRT model is {0}".format(ans))

In [25]:
# export the result as mysubmission2.txt 
PID = np.array(X_test['PID'].values).reshape(len(X_test),1)
Sale_Price = pre_test_y.reshape(len(pre_test_y),1)
ans = np.concatenate((PID,Sale_Price),axis=1)

# save the data with mixted datatype, %d saves PID as integer and %10.5f can save the sale_price as double 
np.savetxt("mysubmission2", ans, delimiter=',', header="PID,Sale_Price", fmt='%d %10.5f', comments='')

### Coordinate Descent lasso algorithm
Use coordinate descent to calculate the result. Unfortunately, I can't get a decent result from this model. My smallest RMSE value is 0.48

In [16]:
# implement Lasso using Coordinate Descent
def one_step_lasso(r, x, lam):
    # x: nx1 matrix
    # r: nx1 matrix
    # Use the soft-thresholding result lasso_j = sgn(βLS_j)(|βLSj|−γ)+
    # return beta_j
    xx = np.sum(np.square(x))
    xr = np.sum(x*r)
    b = (abs(xr) -lam/2)/xx
    b = np.sign(xr)*(b if b>0 else 0) 
    return b

def mylasso(X, y, lam, n_iter, standardize = True):
    # X: n-by-p design matrix without the intercept
    # y: n-by-1 response vector
    # p: p-by-1 vector
    # lam: lambda value
    # n.iter: number of iterations
    # standardize: if True, center and scale X and y. 
    
    #p is the number of features (columns)
    p = X.shape[1]
    
    # n is the number of rows
    n = X.shape[0]
    
    # YOUR CODE
    # If standardize  = TRUE, center and scale X and Y record the corresponding means and sd
    # scaler = StandardScaler()
    # X = scaler.fit_transform(X)
    # y = scaler.fit_transform(y)
    if standardize == True:
        mean_y = np.mean(y)
        mean_X = np.mean(X,axis = 0)
        sd_y = np.std(y)
        sd_X = np.std(X,axis = 0)
        X = (X - np.mean(X,axis = 0))/np.std(X,axis = 0)
        y = (y - np.mean(y))/np.std(y)


    # Initial values for residual and coefficient vector b
    # b: p-by-1 vector, without intercept
        
    b = np.zeros(p)
    r = y

    
    for step in range(n_iter):
        for j in range(p):
        # YOUR CODE 
        # 1) Update the residual vector to be the one
        # in blue on p37 of [lec_W3_VariableSelection.pdf]. 
        # r <-- current residual + X[, j] * b[j]
        # r is n-by-1 vector

            r = r + X[:,j]*b[j]
        # 2) Apply one_step_lasso to update beta_j
        # b[j] = one_step_lasso(r, X[, j], lam)
            b[j] = one_step_lasso(r, X[:,j], lam)

#         # 3) Update the current residual vector
#         # r <-- r - X[, j] * b[j]
            r = r - X[:,j]*b[j]
    # YOUR CODE: scale back b and add intercept b0
    # For b0, check p13 of [lec_W3_VariableSelection.pdf]. 

    b = (sd_y/sd_X)*b
    b_intercept = mean_y - np.dot(mean_X.T,b)
    return (b_intercept,b)

In [17]:
lam = 1000
X = train_X
y = y_train
n_iter = 50
standardize = True
b_intercept = mylasso(X, y, lam, n_iter)[0]
b = mylasso(X, y, lam, n_iter)[1].reshape(264,1)
# np.unique(b)

In [18]:
pre_test_y = np.dot(test_X,b) + b_intercept
ans = round(math.sqrt(np.mean(np.square(np.log(pre_test_y) - np.array(np.log(y_test))))), 4)
print("Root-Mean-Squared-Error (RMSE) for Coordinate decent lasso model is {0}".format(ans))

Root-Mean-Squared-Error (RMSE) for Coordinate decent lasso model is 0.4684


In [24]:
# export the result as mysubmission3.txt
PID = np.array(X_test['PID'].values).reshape(len(X_test),1)
Sale_Price = pre_test_y.reshape(len(pre_test_y),1)
ans = np.concatenate((PID,Sale_Price),axis=1)

# save the data with mixted datatype, %d saves PID as integer and %10.5f can save the sale_price as double 
np.savetxt("mysubmission3", ans, delimiter=',', header="PID,Sale_Price", fmt='%d %10.5f', comments='')